In [0]:
import pyspark
from pyspark.sql import DataFrameWriter
import pandas as pd
from pyspark.sql.functions import *

In [0]:
spark.conf.set("fs.azure.account.key.carehealthcareblobsa.dfs.core.windows.net",
    dbutils.secrets.get(scope="healthcarescope", key="blobaccesskey"))

In [0]:
disease_data = spark.read.csv("abfss://rawhealthcaredata@carehealthcareblobsa.dfs.core.windows.net/disease.csv", header=True, inferSchema=True)

In [0]:
#disease_data.show(10,False)

+---------+----------+------------+
|subgrp_id|disease_id|disease_name|
+---------+----------+------------+
|S101     |110001    |Beriberi    |
|S101     |110002    |Scurvy      |
|S101     |110003    |Goitre      |
|S101     |110004    |Osteoporosis|
|S101     |110005    |Rickets     |
|S101     |110006    |Anaemia     |
|S102     |110007    |Fractures   |
|S102     |110008    |Heart Attack|
|S102     |110009    |Burns       |
|S102     |110010    |Choking     |
+---------+----------+------------+
only showing top 10 rows



In [0]:
#disease_data.groupby(disease_data.columns).count().where("count > 1").show() 

+---------+----------+------------+-----+
|subgrp_id|disease_id|disease_name|count|
+---------+----------+------------+-----+
+---------+----------+------------+-----+



In [0]:
disease_data = disease_data.dropDuplicates()

In [0]:
#disease_data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in disease_data.columns]).show()

+---------+----------+------------+
|subgrp_id|disease_id|disease_name|
+---------+----------+------------+
|        0|         0|           0|
+---------+----------+------------+



In [0]:
#disease_data.show(5,False)

+---------+----------+--------------+
|subgrp_id|disease_id|disease_name  |
+---------+----------+--------------+
|S102     |110012    |Food Poisoning|
|S108     |110044    |Dengue        |
|S108     |110045    |Smallpox      |
|S104     |110020    |Lymphedema    |
|S105     |110030    |Food allergy  |
+---------+----------+--------------+
only showing top 5 rows



In [0]:
#disease_data.select("*").filter(col("subgrp_id").isin("S109")).show(5,False)

+---------+----------+---------------+
|subgrp_id|disease_id|disease_name   |
+---------+----------+---------------+
|S109     |110051    |Phenylketonuria|
|S109     |110054    |Hemophilia     |
|S109     |110050    |Fanconi anaemia|
|S109     |110053    |Rett Syndrome  |
|S109     |110052    |Galactosemia   |
+---------+----------+---------------+
only showing top 5 rows



In [0]:
#disease_data.select("*").filter(col('disease_name') =='Bladder cancer').show(10)

+---------+----------+--------------+
|subgrp_id|disease_id|  disease_name|
+---------+----------+--------------+
|     S107|    110039|Bladder cancer|
+---------+----------+--------------+



In [0]:
output_container_path = "abfss://stagginghealthcaredata@carehealthcareblobsa.dfs.core.windows.net"
output_blob_folder = "stagingdatasets/" 
disease_data.coalesce(1).write.mode("overwrite").option("header", "true").format("com.databricks.spark.csv").save(output_blob_folder)
files = dbutils.fs.ls(output_blob_folder)
output_file = [x for x in files if x.name.startswith("part-")]
dbutils.fs.mv(output_file[0].path, "%s/diseasedatastagging.csv" % output_container_path)

True